# Imports

In [ ]:
!pip install -U albumentations
import os
import numpy as np
from sklearn.model_selection import train_test_split
import random
import sklearn
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from keras import optimizers 
from sklearn.utils import class_weight
from tensorflow.keras.optimizers import SGD, Adam
!pip install opencv-python-headless==4.1.2.30
import cv2
import albumentations as A
from keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import gc
from tensorflow.keras.utils import to_categorical
from random import seed
from random import randint
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from keras.models import model_from_json
from sklearn.metrics import cohen_kappa_score, roc_auc_score, roc_curve,accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Load Data

In [ ]:
import pandas as pd

dataset = pd.read_csv('../input/challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv')
dataset.columns = ['emotion', 'Usage', 'pixels']
test_dataset  = dataset.loc[dataset["Usage"] == 'PublicTest',['emotion','pixels']]
train_dataset = dataset.loc[dataset["Usage"] == 'Training', ['emotion', 'pixels']]
validation_dataset = dataset.loc[dataset["Usage"] == 'PrivateTest', ['emotion', 'pixels']]

# Ploting Data

In [ ]:
print(dataset.shape)
dataset.head()

In [ ]:
validation_dataset.head()

In [ ]:
test_dataset.head()

# Processing Images

In [ ]:
def pixels_to_array(pixels):
    array = np.array(pixels.split(),'uint8')
    return array

def image_reshape(data):
    image = np.reshape(data['pixels'].to_list(),(data.shape[0],48,48,1))
    image = np.repeat(image, 3, -1)
    return image

In [ ]:

train_dataset['pixels'] = train_dataset['pixels'].apply(pixels_to_array)
test_dataset['pixels']  = test_dataset['pixels'].apply(pixels_to_array)
validation_dataset['pixels'] = validation_dataset['pixels'].apply(pixels_to_array)

print("Train:")
print(type(train_dataset['pixels']))
print(train_dataset.shape)

print("Validation:")
print(type(validation_dataset['pixels']))
print(validation_dataset.shape)

print("Test:")
print(type(test_dataset['pixels']))
print(test_dataset.shape)


X_train = image_reshape(train_dataset)
y_train = train_dataset['emotion']
print(X_train.shape)

X_test = image_reshape(test_dataset)
y_test = test_dataset['emotion']
print(X_test.shape)

X_val = image_reshape(validation_dataset)
y_val = validation_dataset['emotion']
print(X_val.shape)
del dataset
gc.collect()

# Data Augmentation

In [ ]:
def get_training_augmentation():
    train_transform = [  
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(scale_limit=0.2, rotate_limit=0.4, shift_limit=0.4, p=1, border_mode= cv2.BORDER_REPLICATE),
        A.Perspective(p=0.6),
        A.Affine(p=0.5, scale= 0.9, rotate = 0.5, fit_output= True),
        A.GaussNoise(p=0.5),
        A.Sharpen(p=0.5),
    ]

    return A.Compose(train_transform)

In [ ]:
def data_augmentate_from_directory(path_folders ,n_generate=1):
  transform = get_training_augmentation()


  for folder in tqdm(path_folders):
    
    caminhos = [os.path.join(folder, nome) for nome in os.listdir(folder)]
    arquivos = [arq for arq in caminhos if os.path.isfile(arq)]
    
    for pathtoimg in arquivos:
      img = cv2.imread(pathtoimg)[:,:,::-1]

    for pathtoimg in arquivos:
      img = cv2.imread(pathtoimg)[:,:,::-1]
      
      for i in range(n_generate): #Valor da quantidade de imagens augmentadas que serão geradas apatir de uma imagem do dataset
        augmentations = transform(image=img)
        augmented_img = augmentations["image"]
        name_img = "Aug_{}.jpg".format(random.randint(1,99999))
        cv2.imwrite(os.path.join(folder,name_img), augmented_img)

# Making Directories for Disk Augmentation

In [ ]:
def PutinDir(Dir, Data, y):
  numbers_name = np.arange(1,1000000)
  for i in range(len(Data)):
    namefile = "Data_"+ str(numbers_name[i]) +".png"
    cv2.imwrite(os.path.join(Dir,str(y[i]),namefile), Data[i])

Delete Folder if Necessary

In [ ]:
!rm -rf ./dataset/

Numpy convert the labels

In [ ]:
y_train, y_val, y_test = np.array(y_train), np.array(y_val), np.array(y_test)

In [ ]:
dataset_dir = "./dataset"
treino_dir = "./dataset/train"
val_dir = "./dataset/validation"
test_dir = "./dataset/test"

os.makedirs(dataset_dir,exist_ok=True)
os.makedirs(treino_dir,exist_ok=True)
os.makedirs(val_dir,exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

labels = [0,1,2,3,4,5,6]

for label in labels:
  os.makedirs(os.path.join(treino_dir,str(label)),exist_ok=True)
  os.makedirs(os.path.join(val_dir,str(label)),exist_ok=True)
  os.makedirs(os.path.join(test_dir,str(label)),exist_ok=True)

numbers_name = np.arange(1,1000000)

PutinDir(treino_dir, X_train, y_train)
PutinDir(val_dir, X_val, y_val)
PutinDir(test_dir, X_test, y_test)

# Ploting Dataset

In [ ]:
def plotDistributionLabels(dataset):
  emotion_prop = (dataset.emotion.value_counts() / len(dataset)).to_frame().sort_index(ascending=True)

  emotions = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

  palette = ['orchid', 'lightcoral', 'orange', 'gold', 'lightgreen', 'deepskyblue', 'cornflowerblue']

  plt.figure(figsize=[12,6])

  plt.bar(x=emotions, height=emotion_prop['emotion'], color=palette, edgecolor='black')
      
  plt.xlabel('Emotion')
  plt.ylabel('Proportion')
  plt.title('Proportion of Emotion Labels')
  plt.show()

In [ ]:
plotDistributionLabels(train_dataset)

In [ ]:
plotDistributionLabels(validation_dataset)

In [ ]:
plotDistributionLabels(test_dataset)

In [ ]:
plt.close()
def plotImagens(dataset, labels):
  plt.rcParams["figure.figsize"] = [16,16]
  emotions = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']
  row = 0
  for emotion in np.unique(labels):

      all_emotion_images = dataset[dataset['emotion'] == emotion]
      for i in range(5):
          
          img = all_emotion_images.iloc[i,].pixels.reshape(48,48)
          lab = emotions[emotion]

          plt.subplot(7,5,row+i+1)
          plt.imshow(img, cmap='binary_r')
          plt.text(-30, 5, s = str(lab), fontsize=10, color='b')
          plt.axis('off')
      row += 5

  plt.show()

In [ ]:
plotImagens(train_dataset, y_train)

In [ ]:
plotImagens(test_dataset, y_test)

# Reader from disk

In [ ]:
def loader(dir_train, dir_val, batch):
  train_datagen = ImageDataGenerator(
    rescale = 1./255
  )

  train_loader = train_datagen.flow_from_directory(
      directory= dir_train,
      target_size=(48, 48),
      color_mode="rgb",
      batch_size=batch,
      class_mode="categorical",
      shuffle=True,
      seed=123456
  )

  val_loader = train_datagen.flow_from_directory(
      directory=dir_val,
      target_size=(48, 48),
      color_mode="rgb",
      batch_size=batch,
      class_mode="categorical",
      shuffle=True,
      seed=123456
  )

  return train_loader, val_loader

In [ ]:
train_loader, valloader = loader("./dataset/train/","./dataset/test/", 64 )

In [ ]:
def plotIterator(train_loader):
  gc.collect()
  x,y= train_loader.next()
  print(x.shape)
  plt.rcParams["figure.figsize"] = [16,16]
  for i in range(0,20):
    plt.subplot(7,5,i+1)
    image = x[i]
    plt.imshow(image)
    plt.axis('off')
  plt.show()

In [ ]:
plotIterator(train_loader)

# Building Model

In [ ]:
def Dense_Model():
  input_shape_densenet = (48, 48, 3)
  densenet_model = keras.applications.DenseNet201(
      include_top=False,
      weights="imagenet",
      input_tensor=None,
      input_shape=input_shape_densenet
  )
  
  densenet_model.trainable = True


  input = keras.Input(shape=(48, 48, 3))
  layer = densenet_model(inputs=input)
  layer = keras.layers.Flatten()(layer)
  layer = keras.layers.Dense(units=512, activation='relu')(layer)
  layer = keras.layers.Dropout(0.4)(layer)
  layer = keras.layers.Dense(units=256, activation='relu')(layer)
  layer = keras.layers.Dropout(0.4)(layer)
  layer = keras.layers.Dense(units=128, activation='relu')(layer)
  layer = keras.layers.Dropout(0.3)(layer)
  output = keras.layers.Dense(units=7, activation='softmax')(layer)

  model = keras.models.Model(inputs=input, outputs=output)
  return model

In [ ]:
model = Dense_Model()
model.summary()

# Training Model Function

In [ ]:
def modelfit(model,trainloader, valloader, epochs,lr, steps, ClassW = False):
  NUM_EPOCHS = epochs 


  #Otimizadores
  adam = Adam(learning_rate=lr) 
  rms = optimizers.rmsprop_v2.RMSProp(lr)
  sgd = optimizers.gradient_descent_v2.SGD(lr)
  adelta = optimizers.adadelta_v2.Adadelta(lr)
  adagrad = optimizers.adagrad_v2.Adagrad(lr)

  model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])
  es = keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                     mode='max',
                                     verbose=1,
                                     patience=7,
                                     min_delta=1e-3,
                                     restore_best_weights=True)
  
  reducelr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                              factor=0.1,
                              patience=2,
                              min_lr=0.000001,
                              mode='min')

  
  if (ClassW):
    class_weights = class_weight.compute_class_weight(class_weight='balanced', classes= np.unique(trainloader.classes), y= trainloader.classes)
    weight = {i : class_weights[i] for i in range(7)} #Converte a lista para um dicionário
    print("Class Weight: ", weight)
  
    history=model.fit(train_loader, 
                      epochs= NUM_EPOCHS,
                      validation_data = valloader,
                      shuffle= True,
                      steps_per_epoch= steps,
                      callbacks=[es,reducelr],
                      class_weight=weight
    )
  else:
    history=model.fit(train_loader, 
                      epochs= NUM_EPOCHS,
                      validation_data = valloader,
                      shuffle= True,
                      steps_per_epoch= steps,
                      callbacks=[es,reducelr]
    )
  return history

# Auxiliary Functions

In [ ]:
def saveModel(model, nameModel, nameWeight):
  model_json = model.to_json()
  with open(nameModel, "w") as json_file:
    json_file.write(model_json)
  # serialize weights to HDF5
  model.save_weights(nameWeight)
  print("Saved model to disk")

def loadModel(nameModel, nameWeight):
  # load json and create model
  json_file = open(nameModel, 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  model = model_from_json(loaded_model_json)
  # load weights into new model
  model.load_weights(nameWeight)
  print("Loaded model from disk")
  return model

In [ ]:
def plot_graphic_history(history):
    plt.figure(figsize=(10, 10))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy in history')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss in history')
    plt.show()

def graphic_acc_loss_per_epochs(lists_history):
  
  for history in lists_history:
    plot_graphic_history(history)

In [ ]:
def test_metrics(model, x,y_true):
  y_pred = modelpred(model,x) # Executa o predict
  draw_confusion_matrix(y_true, y_pred)

In [ ]:
def AugmentateTrain(treino_dir):
  labels = [0,1,2,3,4,5,6]
  paths_train = []
  for label in labels:
    paths_train.append(os.path.join(treino_dir,str(label)))
  data_augmentate_from_directory(paths_train ,n_generate=4)

In [ ]:
import seaborn as sns;
def draw_confusion_matrix(true,preds):
    conf_matx = confusion_matrix(true, preds)
    sns.heatmap(conf_matx, annot=True,annot_kws={"size": 12},fmt='g', cbar=False, cmap="viridis")
    plt.show()
    print(sklearn.metrics.classification_report(true, preds, digits=5))

def modelpred(model,test_full):
  y_get = model.predict(test_full)
  result = []

  for p in y_get:
    result.append(np.argmax(p))

  return result

# Data Augmentation and Train

In [ ]:
def AugmentateTrain_and_especific_class(generalDir, labelsGeneral, XGeneral, labelsEspecif, xEspecif ):
  labels = labelsGeneral
  paths_train = []

  for label in labels:
    paths_train.append(os.path.join(generalDir,str(label)))
  data_augmentate_from_directory(paths_train ,n_generate=XGeneral)
  
  paths_train = []
  for label in labelsEspecif:
    paths_train.append(os.path.join(generalDir,str(label)))

  data_augmentate_from_directory(paths_train ,n_generate=xEspecif)

Augmentate the classes 0,2,3,4,5,6 by 5x and the class 1 by 10x

In [ ]:
model = Dense_Model()
AugmentateTrain_and_especific_class("./dataset/train", [0,2,3,4,5,6], 5, [1], 10)
train_loader, valloader = loader("./dataset/train", "./dataset/validation", 64)
list_model = [] 
list_model.append(modelfit(model,train_loader,valloader, 100, 0.0001, 157981   // 64, ClassW= False))

# Saving the model

In [ ]:
saveModel(model, "./modelAugClass1SW.json", "./modelAugClass1SW.h5")

# Testing the model

In [ ]:
score = model.evaluate(valloader, verbose=1)

In [ ]:
graphic_acc_loss_per_epochs(list_model)

In [ ]:
test_metrics(model, X_test/255, y_test)

# Refine the model

In [ ]:
model = loadModel("./modelAugClass1SW.json", "./modelAugClass1SW.h5")

In [ ]:
model.summary()

In [ ]:
print('Number of layers in base model:', len(model.layers[1].layers), '\n')

print('Names of last ten layers:')
for layer in model.layers[1].layers[-226:]:
    print(layer.name)

Freezing the model and unfreezing only the last 226 layers

In [ ]:
model.trainable = True
for layer in model.layers[1].layers[:-226]:
    layer.trainable = False

model.summary()

Erase the aug dataset and make a new one

In [ ]:
!rm -rf ./dataset/

In [ ]:
dataset_dir = "./dataset"
treino_dir = "./dataset/train"
val_dir = "./dataset/validation"
test_dir = "./dataset/test"

os.makedirs(dataset_dir,exist_ok=True)
os.makedirs(treino_dir,exist_ok=True)
os.makedirs(val_dir,exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

labels = [0,1,2,3,4,5,6]

for label in labels:
  os.makedirs(os.path.join(treino_dir,str(label)),exist_ok=True)
  os.makedirs(os.path.join(val_dir,str(label)),exist_ok=True)
  os.makedirs(os.path.join(test_dir,str(label)),exist_ok=True)

numbers_name = np.arange(1,1000000)

PutinDir(treino_dir, X_train, y_train)
PutinDir(val_dir, X_val, y_val)
PutinDir(test_dir, X_test, y_test)

Augmentate class 0,3,4,5,6 by 2x and class 1, 2 by 4x

In [ ]:
AugmentateTrain_and_especific_class("./dataset/train", [0,3,4,5,6], 2, [1,2], 4)

In [ ]:
train_loader, valloader = loader("./dataset/train", "./dataset/validation", 64)

In [ ]:
list_model = [] 
list_model.append(modelfit(model,train_loader,valloader, 100, 0.00001, 91427   // 64, ClassW= False))
saveModel(model, "modelAugClass1SWFinned.json", "modelAugClass1SWFinned.h5")

In [ ]:
score = model.evaluate(valloader, verbose=1)

In [ ]:
graphic_acc_loss_per_epochs(list_model)

In [ ]:
test_metrics(model, X_test/255, y_test)